# River bifurcation in CONUS workflow
This notebook contains the workflow necessary to extract data from a HUC4 and join it to NABD for bifurcation analysis.

## 1. Install packages

In [1]:
# import sys
# !{sys.executable} -m pip install geofeather
# !{sys.executable} -m pip install nhdnet  #see Setup info document 

## 2. Load modules

In [6]:
#basic analysis 
from pathlib import Path
import os
from time import time
import geopandas as gp
import geofeather
import numpy as np


#modules from SARP analysis
from geofeather import to_geofeather
# from nhdnet.nhd.extract import extract_flowlines 
from nhdnet.nhd.extract_test import extract_flowlines_R  # this function was created by Rachel to extract info from other NHD
# from nhdnet.nhd.extract import extract_waterbodies
from nhdnet.io import serialize_df, serialize_sindex, to_shp 

#Getting the other NHD 
from nhdnet.nhd.download import download_huc4
nhd_dir = Path("data/nhd/source/huc4")


# #pull data from Google Drive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
# drive = GoogleDrive(gauth)

## 3. Initial setup and constants

In [10]:
#Select HUC of interest
HUC2 = 10
i = 19
HUC4 = "{0}{1:02d}".format(HUC2, i)  # this formats the HUC4 name how we want it. ':02d' is string formatting
print(HUC4)
print(type(HUC4))
huc_id = int(HUC4) * 1000000   # the full HUC4 ID
print(huc_id)

# data_dir = Path("data/nhd/source/huc4")  # point to where GDBs are
data_dir = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/GIS/Layers/NHDPlusNationalData")  # point to where GDBs are

#Setting projections
CRS = {           # Using USGS CONUS Albers (EPSG:102003): https://epsg.io/102003  WHY?
    "proj": "aea",
    "lat_1": 29.5,
    "lat_2": 45.5,
    "lat_0": 37.5,
    "lon_0": -96,
    "x_0": 0,
    "y_0": 0,
    "datum": "NAD83",
    "units": "m",
    "no_defs": True,
}
print(data_dir)

1019
<class 'str'>
1019000000
/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/GIS/Layers/NHDPlusNationalData


## 4. Read in the geodatabase

In [ ]:
# sys.path.append('/Users/rachelspinti/Documents/River_bifurcation/data/nhd/source/huc4/1019') #call where these scripts are located

# gdb = data_dir/HUC4/ "NHDPLUS_H_{HUC4}_HU4_GDB.gdb".format(HUC4=HUC4)
# print(gdb)
# read_start = time()
# flowlines = extract_flowlines(gdb, target_crs=CRS)
# print("Read {:,} flowlines in  {:.0f} seconds".format(len(flowlines), time() - read_start))

gdb = data_dir/"NHDPlusV21_National_Seamless_Flattened_Lower48.gdb"
print(gdb)
read_start = time()
flowlines = extract_flowlines_R(gdb, target_crs=CRS)
print("Read {:,} flowlines in  {:.0f} seconds".format(len(flowlines), time() - read_start))

# gdb = data_dir/HUC4/ "NHDPLUS_H_{HUC4}_HU4_GDB.gdb".format(HUC4=HUC4)
# print(gdb)
# read_start = time()
# flowlines, joins = extract_flowlines(gdb, target_crs=CRS)
# print("Read {:,} flowlines in  {:.0f} seconds".format(len(flowlines), time() - read_start))

/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/GIS/Layers/NHDPlusNationalData/NHDPlusV21_National_Seamless_Flattened_Lower48.gdb
Reading flowlines


### 4.1 Read in the HUC4 shapefile (1019) I made 

In [5]:
huc_test = gp.read_file('/Users/rachelspinti/Desktop/HUC_test/Test1029.shp') # this is actually HUC 1019
print(huc_test)

# NABD = gp.read_file('/Users/rachelspinti/Documents/River_bifurcation/data/nabd/nabd_fish_barriers_2012.shp')

        OBJECTID      COMID                    FDATE RESOLUTION GNIS_ID  \
0      1518650.0    5239914  1999/06/01 00:00:00.000     Medium  183410   
1      1518651.0    5239908  1999/06/01 00:00:00.000     Medium  183410   
2      1518652.0    5239882  1999/06/01 00:00:00.000     Medium  183410   
3      1518653.0    5239880  1999/06/01 00:00:00.000     Medium  183410   
4      1518654.0    5239854  1999/06/01 00:00:00.000     Medium  183410   
...          ...        ...                      ...        ...     ...   
14842  1533492.0  940190113  2010/12/01 00:00:00.000     Medium    None   
14843  1533493.0  940190138  2009/03/27 00:00:00.000     Medium    None   
14844  1533494.0  940190275  2009/09/22 00:00:00.000     Medium    None   
14845  1533495.0  940190103  2010/12/01 00:00:00.000     Medium    None   
14846  1533496.0   17451615  2005/08/30 00:00:00.000     Medium  202890   

             GNIS_NAME  LENGTHKM       REACHCODE         FLOWDIR  WBAREACOMI  \
0       Tarryall Cr

## 5. Join the two datasets
Check this link out for help: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/vector-data-processing/spatial-joins-in-python-geopandas-shapely/

In [ ]:
#Spatial join with geopandas
gp.sjoin(flowlines, huc_test)  # join by attribute: is that spatial join?

## Getting information about what came out of this

First for the flowlines -- this is a geodataframe with the flowline geometry. Comes from *flowlines.py*

In [ ]:
type(flowlines)
flowlines.head(3)
# print(flowlines.describe)
# flowlines.plot()
# print(flowlines.shape)
# print(list(flowlines.columns))
# flowlines[flowlines.streamorder>6]
# flowlines[flowlines.streamorder>6].plot()

Then for the joins - this is a dataframe with the linkage information. Comes from *flowlines.py*

In [ ]:
type(joins)
joins.head(3)

# joins.plot()
# joins[joins.downstream_id>0].plot()  # plotting test
# print(joins.describe)

## Reorganizing the columns (not really sure why they do this)

In [ ]:
flowlines= flowlines[["geometry",
                 "lineID",
                 "NHDPlusID",
                "ReachCode",
                "FType",
                "length",
                "sinuosity",
                "sizeclass",
                "streamorder"]]
print(flowlines.shape)
#print(max(flowlines['NHDPlusID']), min(flowlines['NHDPlusID']))
print(max(flowlines['lineID']), min(flowlines['lineID']))



### Compare flowlines and joins
The lineIDs are created in *flowlines.py* 

In [ ]:
from IPython.core.display import HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

# Calculate lineIDs to be unique across the regions
#LC - .loc Accesses a group of rows and columns by label(s) or a boolean array
flowlines["lineID"] += huc_id
# flowlines.head(3)

# Set updated lineIDs with the HUC4 prefix
joins.loc[joins.upstream_id != 0, "upstream_id"] += huc_id
joins.loc[joins.downstream_id != 0, "downstream_id"] += huc_id
# joins.head(3)

multi_table([flowlines.head(3), joins.head(3)])

## Need to figure out what the read water bodies part is doing--- that function doesn't work in the sourced library but exists in the git repo
Check if we need to have the water bodies in order to have a fully connected drainage network or not.

*The joins are the connections between the flowlines, so do not need waterbodies. See extract.py*

In [ ]:
### Read waterbodies
read_start = time()
waterbodies = extract_waterbodies(
                gdb,
                target_crs=CRS,
                exclude_ftypes=WATERBODY_EXCLUDE_FTYPES,
                min_area=WATERBODY_MIN_SIZE)

print("Read {:,} waterbodies in  {:.0f} seconds".format(
                    len(waterbodies), time() - read_start))

# calculate ids to be unique across region
waterbodies["wbID"] += huc_id

### Only retain waterbodies that intersect flowlines
print("Intersecting waterbodies and flowlines")
wb_joins = gp.sjoin(waterbodies, flowlines, how="inner", op="intersects")[["wbID", "lineID"]]

waterbodies = waterbodies.loc[waterbodies.wbID.isin(wb_joins.wbID)].copy()
# print("Retained {:,} waterbodies that intersect flowlines".format(
#                     len(waterbodies))

## Getting rid of dead ends
Note in this example there are none so nothing changes
~ means take the compliment

In [ ]:
print(joins.shape)
joins=joins.loc[~((joins.downstream == 0) & (joins.upstream == 0))].copy()
print(joins.shape)

# Serializing the flowlines 
I think this means that the data structure is changed. So go from dataframe to feather file because it is easier to work with.

Blog post on to_geofeather: https://medium.com/@brendan_ward/introducing-geofeather-a-python-library-for-faster-geospatial-i-o-with-geopandas-341120d45ee5 
reset_index explanation: https://www.geeksforgeeks.org/reset-index-in-pandas-dataframe/

In [ ]:
print("serializing {:,} flowlines to feather".format(len(flowlines)))
region_dir=data_dir/HUC4/ "NHDPLUS_H_{HUC4}_HU4_GDB.gdb".format(HUC4=HUC4)
# region_dir=Path(HUC4)
flowlines = flowlines.reset_index(drop=True)
to_geofeather(flowlines, region_dir /"flowlines.feather")
# #Serializes a pandas DataFrame to a feather file on disk --- just writing it efficiently
serialize_df(joins,  "flowline_joins.feather", index=False)


## Not part of the workflow just testing out joins to see how they made that table¶
This is copied from extract.py. I think the reson we don't get the same downstream/upstream_ids is the filtering they do with coastlines and the removed_idx

In [ ]:
#     print("Filtering out loops and coastlines")
#     coastline_idx = flowlines.loc[(flowlines.FType == 566)].index
#     removed_idx = flowlines.loc[
#         (flowlines.streamorder != flowlines.StreamCalc) | (flowlines.FlowDir.isnull()) | (flowlines.FType == 566)
#     ].index
#     flowlines = flowlines.loc[~flowlines.index.isin(removed_idx)].copy()
#     print("{:,} features after removing loops and coastlines".format(len(flowlines)))

In [ ]:
print("Reading flowline joins")

#this line reads the flowlines and grabs out just the columns 'FromNHDPID' and 'ToNHDPID' then it renames them as upstream and downstream
join_df = gp.read_file(gdb, layer="NHDPlusFlow")[["FromNHDPID", "ToNHDPID"]].rename(columns={"FromNHDPID": "upstream", "ToNHDPID": "downstream"})
join_df.upstream = join_df.upstream.astype("uint64")
join_df.downstream = join_df.downstream.astype("uint64")

join_df = join_df.drop_duplicates()
join_df = (join_df.join(flowlines.lineID.rename("upstream_id"), on="upstream").
          join(flowlines.lineID.rename("downstream_id"), on="downstream")
          .fillna(0))

for col in ("upstream", "downstream"):
        join_df[col] = join_df[col].astype("uint64")

for col in ("upstream_id", "downstream_id"):
        join_df[col] = join_df[col].astype("uint32")

# test=flowlines[0:3]
#print(test)
#print(test.FType)
#test.FType.rename("testing")
# print(test)
print(join_df)


In [ ]:
joins.head(3)
# joins.describe

In [ ]:
#grabbing two columns out
print(join_df.shape)
# test=join_df[["FromNHDPID", "ToNHDPID"]]
test=join_df[["upstream_id", "downstream_id"]]
print(test.shape)
test.head(3)

In [ ]:
# #Grabbing two columns out and modifying 

# test2=join_df[["FromNHDPID", "ToNHDPID"]].rename(columns={"FromNHDPID": "upstream", "ToNHDPID": "downstream"})
# test2.head(3)

## Reading in the NABD shape file
Useful tips on working with shape files: https://www.earthdatascience.org/workshops/gis-open-source-python/intro-vector-data-python/

In [ ]:
NABD = gp.read_file('/Users/rachelspinti/Documents/River_bifurcation/data/nabd/nabd_fish_barriers_2012.shp')

In [ ]:
#look at the properties

In [ ]:
print(NABD.shape)
print(list(NABD.columns))
NABD.head(3)

## Attempt at spatial join of NHD_HUC4 and NHD I have
#### 1. Filter NHD

In [ ]:
# Stuck here...
# Need a script like the extract.py but that is separate so we can extract COMID, so copy the code, but save elsewhere?

#### 2. Join NHD_HUC4 and NHD
Check this link out for help: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/vector-data-processing/spatial-joins-in-python-geopandas-shapely/